# FSC curve of empiar10311_stack_crop.mrc VS several sigma values in Stucture Preserving Gaussian Denoising

In [ ]:
import mrcfile
import matplotlib.pyplot as plt
from self_fourier_shell_correlation import fsc_utils as fsc # pip install --upgrade --force-reinstall "self_fourier_shell_correlation @ git+https://github.com/vicente-gonzalez-ruiz/self_fourier_shell_correlation"
from denoising.volume.gaussian import Monochrome_Denoising as GD # pip install --upgrade --force-reinstall "denoising @ git+https://github.com/vicente-gonzalez-ruiz/denoising"
from denoising.volume.OF_gaussian import Monochrome_Denoising as SPGD # pip install --upgrade --force-reinstall "denoising @ git+https://github.com/vicente-gonzalez-ruiz/denoising"
from denoising.utils import get_gaussian_kernel # pip install --upgrade --force-reinstall "denoising @ git+https://github.com/vicente-gonzalez-ruiz/denoising"
import numpy as np
import logging

In [ ]:
vol_name = "empiar10311_stack_crop"
vol_extension = ".mrc"
vols_path = "/mnt2/TomogramDenoising/vols/"

In [ ]:
from collections import namedtuple # Comes with the PSL
Args = namedtuple("args", ["vol"])
args = Args(vols_path + vol_name + vol_extension)

In [ ]:
with mrcfile.open(args.vol, permissive=True) as mrc:
    vol= mrc.data

In [ ]:
vol.shape

In [ ]:
vol = vol[0:100, 0:100, 0:100]

In [ ]:
Z_dim = vol.shape[0]
Z2 = Z_dim//2
#plt.imshow(vol[Z2], cmap="gray")

In [ ]:
def get_kernel_coeffs(sigma):
    sigmas = np.array([sigma, sigma, sigma])
    kernel = [None]*3
    kernel[0] = get_gaussian_kernel(sigmas[0])
    kernel[1] = get_gaussian_kernel(sigmas[1])
    kernel[2] = get_gaussian_kernel(sigmas[2])
    return kernel

In [ ]:
sigma = 2.0
kernel = get_kernel_coeffs(sigma)

In [ ]:
logging.basicConfig(format="[%(filename)s:%(lineno)s %(funcName)s()] %(message)s")
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

In [ ]:
denoiser = GD(logger)
vol_denoised = denoiser.filter(vol, kernel)

In [ ]:
#plt.imshow(vol_denoised[Z2], cmap="gray")

In [ ]:
freq, c_GD = fsc.get_FSC_curve(vol, vol_denoised)

In [ ]:
denoiser = SPGD(logger)
vol_denoised = denoiser.filter(vol, kernel, l=2, w=5)
freq, c_SPGD = fsc.get_FSC_curve(vol, vol_denoised)

In [ ]:
#plt.imshow(vol_denoised[Z2], cmap="gray")

In [ ]:
plt.title(vol_name)
plt.xlabel("Spatial Frequency")
plt.ylabel("Fourier Shell Correlation")
plt.plot(freq, c_GD, label=f"GD")
plt.plot(freq, c_SPGD, label=f"SPGD")
plt.legend()
plt.savefig(vol_name + "_GD_vs_SPGD" + ".svg")
plt.show()

In [ ]:
FSC_threshold = 0.5

In [ ]:
np.sum(c_GD[c_GD > FSC_threshold])

In [ ]:
np.sum(c_SPGD[c_SPGD > FSC_threshold])